In [4]:
import os
import pandas as pd
from docxtpl import DocxTemplate
from docx2pdf import convert
import random

In [ ]:
import os
print(os.path.expanduser("~"))
# Python内置函数，获取当前用户主目录

C:\Users\heart


In [22]:
def generate_application_documents():

    '''
    # Step 1: 创建输出目录
    home_dir = os.path.expanduser("~")
    output_dir = os.path.join(home_dir, "HW_School_Application")
    # os.path.join() 把主目录路径和子目录拼接，形成完整路径
    os.makedirs(output_dir, exist_ok=True)
    # makedirs 创建目录
    '''
    output_dir=r"D:\大三上\人工智能与python\第三次作业\HW_School_Application"

    # Step 2: 读取数据文件
    try:
        # 读取大学数据
        university_df = pd.read_excel('University.xlsx', header=None) # 否则默认, header=0第一行是表头，会跳过数据
        # .astype(str)强制转换为字符串类型
        universities = university_df.iloc[:, 2].astype(str).tolist()  # 返回第三列 .tolist()转换为列表List['Harvard University', ...]
        # .iloc通过行和列的整数位置进行切片、过滤和索引 ，例如：选择第2~第3行，第1~第2列 df.iloc[1:3, 0:2]

        # 读取专业领域数据
        area_df = pd.read_excel('Area.xlsx')
        #area_df['Area'] = area_df['Area'].astype(str)
        # ==============================================================

    except FileNotFoundError as e:
        print(f"错误：找不到数据文件 - {e}")
        return
    
    # Step 3: 准备数据映射
    area_journals = {} # 字典
    all_jobs = []
    all_skills = []
    
    for _, row in area_df.iterrows():
        # _代表该变量不关心，不会使用
        #iterrows() 按行迭代，每次回一个元组(index, row_data)

        area = row['Area']
        area_journals[area] = [row['J1'], row['J2'], row['J3']] # 键是 area，值是列表，包含当前行的J1、J2、J3
        # 例如：'Finance': ['Journal of Finance', 'Journal of Financial Economics', 'Review of Financial Studies']

        # 处理职业数据（每行一个职业，累积到列表中）
        job = str(row['Job']).strip()
        if job != 'nan' and job not in all_jobs:
            all_jobs.append(job)
    
        # 技能
        skill = str(row['Skills']).strip()
        if skill and skill != 'nan' and skill not in all_skills:
                all_skills.append(skill)
    
    # Step 4: 生成文档
    generated_files = []
    
    for area in area_journals.keys(): #遍历每个领域
        for university in universities: # 遍历每个大学

            # context字典存储template中所有需要替换的变量和对应的值
            context = {
                'area': area,
                'university': university,
                'J1': area_journals[area][0],
                'J2': area_journals[area][1],
                'J3': area_journals[area][2],
                'job': random.choice(all_jobs)
            }
            
            # 随机选择5个技能
            available_skills = all_skills.copy() # 创建技能的副本，避免修改原始数据
            selected_skills = []
            for i in range(5):
                if available_skills: #循环5次，选择5个技能
                    skill = random.choice(available_skills)
                    selected_skills.append(skill)
                    available_skills.remove(skill) # 避免重复选择
            
            # 添加技能到上下文,循环5次，为 skill1~skill5 赋值
            for i in range(5):
                context[f'skill{i+1}'] = selected_skills[i] 
                # 左边 f'skill{i+1}' 是字典键名,f-string（格式化字符串）i从0开始

            # Step 5: 生成Word文档
            try:
                doc = DocxTemplate("templateHWX.docx") # 加载模板文件
                doc.render(context)  #用context字典中的值替换模板中的 {{变量}}
                
                # 生成文件名,如：Application_Finance_Harvard_University
                # 以防万一用.replace(' ', '_') 将空格换为下划线，避免文件名出错
                filename = f"Application_{area.replace(' ', '_')}_{university.replace(' ', '_')}"
                
                word_file = os.path.join(output_dir, f"{filename}.docx") #组合成完整文件路径
                doc.save(word_file) # 保存Word文档
                generated_files.append(word_file) #记录文件路径
                print(f"生成文件: {word_file}")
                
            except Exception as e:
                #print(f"生成Word文档时出错: {e}") 
                # try-except 块确保单个文件生成失败不会影响其他文件
                # 出错时打印错误信息并 continue 继续下一个循环
                continue
    
    # Step 6: 生成PDF
    try:
        if os.name == 'nt':  # 'nt' 代表Windows系统
            for word_file in generated_files[:1]:  # 只转换第一个文件作为示例
                word_file_str = str(word_file) # 将WindowsPath转换为字符串，然后再替换扩展名
                pdf_file = word_file.replace('.docx', '.pdf')
                #—————————————————————————这里的 word_file 是个 WindowsPath 对象（因为用了os.path.join），不是普通的字符串。WindowsPath 对象没有 replace 方法

                convert(word_file_str, pdf_file) #调用 docx2pdf 库的转换函数
                print(f"生成PDF文件: {pdf_file}")
        else:
            print("非Windows系统，跳过PDF生成")
    except Exception as e:
        print(f"生成PDF时出错: {e}")
    
    # Step 7: 显示完成信息
    print(f"\n完成！共生成 {len(generated_files)} 个申请文件")
    print(f"文件保存在: {output_dir}")
    print("\n请将以下文件上传到GitHub:")
    print("1. 此Python代码文件")
    print("2. University.xlsx")
    print("3. Area.xlsx") 
    print("4. template.docx")
    print("5. 任意一个生成的PDF或Word文件")

if __name__ == "__main__":
    # Python程序的入口
    generate_application_documents()

生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_Harvard_University.docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_University_of_California-Berkeley.docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_University_of_Chicago.docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_Princeton_University.docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_Stanford_University.docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_Toulouse_School_of_Economics_(TSE).docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_Columbia_University.docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_New_York_University_(NYU).docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_University_of_Michigan.docx
生成文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Applica

  0%|          | 0/1 [00:00<?, ?it/s]

生成PDF文件: D:\大三上\人工智能与python\第三次作业\HW_School_Application\Application_Economics_Harvard_University.pdf

完成！共生成 90 个申请文件
文件保存在: D:\大三上\人工智能与python\第三次作业\HW_School_Application

请将以下文件上传到GitHub:
1. 此Python代码文件
2. University.xlsx
3. Area.xlsx
4. template.docx
5. 任意一个生成的PDF或Word文件
